In [ ]:
# week 1
!pip install PyPDF2 spacy pandas
!python -m spacy download en_core_web_sm
!pip install PyPDF2 spacy pandas requests
!python -m spacy download en_core_web_sm
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 63.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 106.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


USING API to look for papers online

In [ ]:
# Install required libraries
!pip install pymed arxiv semanticscholar

import sqlite3
import pandas as pd
from pymed import PubMed
import arxiv
from semanticscholar import SemanticScholar
import re
from datetime import datetime

# Function to clean text (simple preprocessing)
def clean_text(text):
    if not text or pd.isna(text):
        return ""
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip().lower()

# Function to initialize or update the research_papers table
def initialize_db(db_name="healthcare_research.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    # Drop the existing table and recreate with full schema
    cursor.execute("DROP TABLE IF EXISTS research_papers")
    cursor.execute('''
        CREATE TABLE research_papers (
            filename TEXT PRIMARY KEY,
            cleaned_text TEXT,
            title TEXT,
            source TEXT,
            publication_date TEXT
        )
    ''')
    conn.commit()
    conn.close()
    print("Initialized research_papers table with updated schema.")

# Function to fetch and store papers
def collect_research_papers(db_name="healthcare_research.db", max_per_source=20):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Initialize counters
    total_papers = 0

    # --- PubMed ---
    pubmed = PubMed(tool="ResearchPaperCollector", email="your_email@example.com")  # Replace with your email
    query = "healthcare innovation"
    results = pubmed.query(query, max_results=max_per_source)
    for article in results:
        title = article.title or "Untitled"
        abstract = article.abstract or ""
        pub_date = article.publication_date.strftime('%Y-%m-%d') if article.publication_date else "Unknown"
        filename = f"pubmed_{article.pubmed_id}"
        cleaned_text = clean_text(abstract)  # Full text not available via API, using abstract
        try:
            cursor.execute('''
                INSERT OR IGNORE INTO research_papers (filename, cleaned_text, title, source, publication_date)
                VALUES (?, ?, ?, ?, ?)
            ''', (filename, cleaned_text, title, "PubMed", pub_date))
            total_papers += 1
            print(f"Added {filename} from PubMed")
        except sqlite3.IntegrityError:
            print(f"Skipping duplicate {filename}")

    # --- arXiv ---
    arxiv_client = arxiv.Client()
    search = arxiv.Search(
        query="healthcare",
        max_results=max_per_source,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    for result in arxiv_client.results(search):
        title = result.title or "Untitled"
        summary = result.summary or ""
        pub_date = result.published.strftime('%Y-%m-%d') if result.published else "Unknown"
        filename = f"arxiv_{result.entry_id.split('/')[-1]}"
        cleaned_text = clean_text(summary)  # Full text not directly available, using summary
        try:
            cursor.execute('''
                INSERT OR IGNORE INTO research_papers (filename, cleaned_text, title, source, publication_date)
                VALUES (?, ?, ?, ?, ?)
            ''', (filename, cleaned_text, title, "arXiv", pub_date))
            total_papers += 1
            print(f"Added {filename} from arXiv")
        except sqlite3.IntegrityError:
            print(f"Skipping duplicate {filename}")

    # --- Semantic Scholar ---
    sch = SemanticScholar()
    search = sch.search_paper(query="healthcare innovation", limit=max_per_source)
    for paper in search:
        title = paper.title or "Untitled"
        abstract = paper.abstract or ""
        pub_date = paper.year if paper.year else "Unknown"
        filename = f"semanticscholar_{paper.paperId}"
        cleaned_text = clean_text(abstract)  # Full text not available, using abstract
        try:
            cursor.execute('''
                INSERT OR IGNORE INTO research_papers (filename, cleaned_text, title, source, publication_date)
                VALUES (?, ?, ?, ?, ?)
            ''', (filename, cleaned_text, title, "Semantic Scholar", str(pub_date)))
            total_papers += 1
            print(f"Added {filename} from Semantic Scholar")
        except sqlite3.IntegrityError:
            print(f"Skipping duplicate {filename}")

    # Commit and close
    conn.commit()
    conn.close()
    print(f"Collected and stored {total_papers} papers in total.")

# Main execution
if __name__ == "__main__":
    initialize_db()
    collect_research_papers(max_per_source=20)  # 20 from each source = 60 total

Initialized research_papers table with updated schema.
Added pubmed_40327459 from PubMed
Added pubmed_40327390 from PubMed
Added pubmed_40327384 from PubMed
Added pubmed_40327329 from PubMed
Added pubmed_40327286
27918725
33766750
17537614
20668052
33754023
19641587
35210368
17384583
9446642
32392088
34753510
35134463
33045622
24382354
21440622
23474486
35273493
30053525
38574433
37154056
26232232
33314649
32468629
11709088
37425037
23325925
10628339
36435202
32112883
24058702
31883358
38323433
26112889
17250813
37871668
33437153
38150302
38316068
29061338
27567406
35914558
37714227
36510036
32958254
29304479 from PubMed
Added pubmed_40327095
32984198
31953750
15955855
10352583
16615053
23958828
14531007
29306987
12677511
21643638
20093572
19022993
21567143
26736224
15114491
23285596
22764084
27330520
14974593
18400271
15476531
23793577
19255628
32613678 from PubMed
Added pubmed_40327021 from PubMed
Added pubmed_40326881 from PubMed
Added pubmed_40326805 from PubMed
Added pubmed_403267

In [ ]:
# week 1
import os
import re
import sqlite3
import pandas as pd
import PyPDF2
import spacy
import gdown
import zipfile
from spacy.lang.en.stop_words import STOP_WORDS

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")
# Google Drive file URL and local paths
GOOGLE_DRIVE_URL = "https://drive.google.com/uc?id=1QfIv0MnqHNOzKZqXSWgRU_SjlnGyx2vj"
ZIP_FILE = "med_articles2.zip"
WORKING_DIR = "med_articles_temp"
os.makedirs(WORKING_DIR, exist_ok=True)

# Step 1: Download and Extract ZIP
def download_and_extract_zip():
    if not os.path.exists(ZIP_FILE):
        print(f"Downloading {ZIP_FILE} from Google Drive...")
        gdown.download(GOOGLE_DRIVE_URL, ZIP_FILE, quiet=False)
    else:
        print(f"{ZIP_FILE} already exists, skipping download.")

    try:
        with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
            print(f"Extracting {ZIP_FILE} to {WORKING_DIR}...")
            zip_ref.extractall(WORKING_DIR)
        print(f"Successfully extracted to {WORKING_DIR}")
    except zipfile.BadZipFile:
        print(f"Error: {ZIP_FILE} is not a valid ZIP file or is corrupted.")
        raise

# Step 2: Text Extraction and Metadata Parsing
def extract_text_from_pdf(pdf_path):
    print(f"Attempting to extract text from {pdf_path}")
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() or ""
            print(f"Extracted {len(text)} characters from {pdf_path}")
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

def parse_metadata(raw_text, filename):
    doc = nlp(raw_text[:2000])
    title = "Unknown"
    author = "Unknown"
    pub_date = "Unknown"
    abstract = "Unknown"

    lines = raw_text.split('\n')
    for line in lines[:10]:
        if re.search(r"^(Title:|Abstract|Authors?:)", line, re.IGNORECASE):
            if "Title:" in line:
                title = re.sub(r"Title:\s*", "", line, flags=re.IGNORECASE).strip()
            elif "Author" in line:
                author = re.sub(r"Authors?:\s*", "", line, flags=re.IGNORECASE).strip()
        elif len(line.strip()) > 10 and title == "Unknown":
            title = line.strip()
            break

    for sent in doc.sents:
        if "by" in sent.text.lower() or re.search(r"[A-Z][a-z]+,\s*[A-Z]", sent.text):
            author = sent.text.strip()
            break

    date_match = re.search(r"\b(19|20)\d{2}\b", raw_text)
    if date_match:
        pub_date = date_match.group(0)

    abstract_match = re.search(r"Abstract\s*[:\-\s]*(.+?)(?:\n\n|\r\n\r\n|Introduction|Methods|$)", raw_text, re.DOTALL | re.IGNORECASE)
    if abstract_match:
        abstract = abstract_match.group(1).strip()

    if title == "Unknown":
        title = os.path.splitext(filename)[0]

    return {"title": title, "author": author, "pub_date": pub_date, "abstract": abstract}

# Step 3: Preprocessing
def preprocess_text(text):
    doc = nlp(text[:100000])
    tokens = [token.text.lower() for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

# Step 4: Process Files and Store in DB
def process_files_and_store_in_db(directory):
    conn = sqlite3.connect("healthcare_research.db")
    cursor = conn.cursor()

    # Add author and other columns if they don't exist
    cursor.execute("PRAGMA table_info(research_papers)")
    columns = [col[1] for col in cursor.fetchall()]
    if "author" not in columns:
        cursor.execute("ALTER TABLE research_papers ADD COLUMN author TEXT")
    if "pub_date" not in columns:
        cursor.execute("ALTER TABLE research_papers ADD COLUMN pub_date TEXT")
    if "abstract" not in columns:
        cursor.execute("ALTER TABLE research_papers ADD COLUMN abstract TEXT")
    if "cleaned_text" not in columns:
        cursor.execute("ALTER TABLE research_papers ADD COLUMN cleaned_text TEXT")
    conn.commit()



    cursor.execute('''
        CREATE TABLE IF NOT EXISTS research_papers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT,
            title TEXT,
            author TEXT,
            pub_date TEXT,
            abstract TEXT,
            cleaned_text TEXT
        )
    ''')

    data = {
        "filename": [], "title": [], "author": [], "pub_date": [], "abstract": [], "cleaned_text": []
    }

    print(f"Searching recursively for PDFs in {directory}")
    pdf_count = 0
    for root, _, files in os.walk(directory):  # Recursive search
        for filename in files:
            if not filename.endswith(".pdf"):
                print(f"Skipping {filename} - not a PDF")
                continue

            pdf_count += 1
            file_path = os.path.join(root, filename)
            raw_text = extract_text_from_pdf(file_path)

            if not raw_text:
                print(f"Skipping {filename} - no text extracted")
                continue

            metadata = parse_metadata(raw_text, filename)
            cleaned_text = preprocess_text(raw_text)

            data["filename"].append(filename)
            data["title"].append(metadata["title"])
            data["author"].append(metadata["author"])
            data["pub_date"].append(metadata["pub_date"])
            data["abstract"].append(metadata["abstract"])
            data["cleaned_text"].append(cleaned_text)

            cursor.execute('''
                INSERT INTO research_papers (filename, title, author, pub_date, abstract, cleaned_text)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (filename, metadata["title"], metadata["author"], metadata["pub_date"], metadata["abstract"], cleaned_text))
            print(f"Inserted {filename} into database")

    print(f"Found and processed {pdf_count} PDF files")
    conn.commit()

    cursor.execute("SELECT COUNT(*) FROM research_papers")
    total_rows = cursor.fetchone()[0]
    print(f"Total records in database: {total_rows}")

    cursor.execute("SELECT * FROM research_papers LIMIT 5")
    rows = cursor.fetchall()
    print("Sample data from database:")
    for row in rows:
        print(row)

    conn.close()

    df = pd.DataFrame(data)
    df.to_csv("preprocessed_data.csv", index=False)
    print("Data also saved to preprocessed_data.csv")

# Run the script
if __name__ == "__main__":
    download_and_extract_zip()
    process_files_and_store_in_db(WORKING_DIR)

med_articles2.zip already exists, skipping download.
Extracting med_articles2.zip to med_articles_temp...
Successfully extracted to med_articles_temp
Searching recursively for PDFs in med_articles_temp
Attempting to extract text from med_articles_temp/articles medical/07-1411.pdf
Extracted 4932 characters from med_articles_temp/articles medical/07-1411.pdf
Inserted 07-1411.pdf into database
Attempting to extract text from med_articles_temp/articles medical/Longest-2287_Chap1-e8ed4666.pdf
Extracted 126206 characters from med_articles_temp/articles medical/Longest-2287_Chap1-e8ed4666.pdf
Inserted Longest-2287_Chap1-e8ed4666.pdf into database
Attempting to extract text from med_articles_temp/articles medical/POST-PN-0545.pdf
Extracted 28237 characters from med_articles_temp/articles medical/POST-PN-0545.pdf
Inserted POST-PN-0545.pdf into database
Attempting to extract text from med_articles_temp/articles medical/Medical Waste Treatment and Disposal Methods Used by Hospitals in Oregon  Was

checking The dabase

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("healthcare_research.db")
cursor = conn.cursor()

# Get the total number of rows
cursor.execute("SELECT COUNT(*) FROM research_papers")
total_rows = cursor.fetchone()[0]
print(f"Total number of records: {total_rows}")

# View the first 5 rows
cursor.execute("SELECT * FROM research_papers LIMIT 20")
rows = cursor.fetchall()

print("\nFirst 20 records:")
for row in rows:
    print(row)

# View column names
cursor.execute("PRAGMA table_info(research_papers)")
columns = cursor.fetchall()
print("\nColumn names:")
for col in columns:
    print(col[1])  # col[1] is the column name

# Close the connection
conn.close()

Total number of records: 1074

First 20 records:
('pubmed_40327459', 'to evaluate prospective changes in psychological function among menopausal women using continuous nitroglycerin for hot flashes and examine repeated associations between psychological symptoms and hot flashes menopausal women with 7 hot flashesday were randomized to continuous transdermal nitroglycerin 0204 mgh or placebo for 12 weeks psychological function was evaluated using the center for epidemiologic studiesdepression scale cesd generalized anxiety disorders7 gad7 and menopausespecific quality of life menqol psychosocial subscale linear mixed models examined treatment effects on mood symptoms from baseline to 5 and 12 weeks based on validated 7day hot flash diaries additional models examined associations between mood symptoms and hot flash frequency in both groups combined among the 141 randomized participants mean 10835 hot flashesday nitroglycerin therapy did not improve cesd or gad7 scores prespecified second

week 2 new with bert

In [ ]:
# weeeek 2 new
# Week 2 - Extractive Summarization using BERT

import sqlite3
import pandas as pd

# Ensure necessary packages are installed
try:
    from summarizer import Summarizer
except ImportError:
    print("Installing necessary packages...")
    import os
    os.system("pip install bert-extractive-summarizer")
    from summarizer import Summarizer

# Step 1: Set up the BERT Extractive Summarization model
print("Loading BERT summarization model...")
bert_model = Summarizer()

# Step 2: Connect to the SQLite database
def connect_to_db():
    conn = sqlite3.connect("healthcare_research.db")
    return conn

# Step 3: Add a summary column to the database (if not already present)
def add_summary_column(conn):
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(research_papers)")
    columns = [col[1] for col in cursor.fetchall()]
    if "summary" not in columns:
        cursor.execute("ALTER TABLE research_papers ADD COLUMN summary TEXT")
        print("Added 'summary' column to research_papers table.")
    conn.commit()

# Step 4: Generate extractive summaries using BERT
def generate_summary(text, ratio=0.2):  # Extract 20% of the most important sentences
    try:
        if len(text.split()) < 50:  # Skip very short texts
            return "Text too short for summarization"
        summary = bert_model(text, ratio=ratio)
        return summary
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return "Summary generation failed"

# Step 5: Process papers and update the database
def summarize_papers():
    conn = connect_to_db()
    add_summary_column(conn)
    cursor = conn.cursor()

    # Check if an id column exists; if not, add it as a non-primary key column
    cursor.execute("PRAGMA table_info(research_papers)")
    columns = [col[1] for col in cursor.fetchall()]
    if "id" not in columns:
        cursor.execute("ALTER TABLE research_papers ADD COLUMN id INTEGER")  # Removed PRIMARY KEY and AUTOINCREMENT
        # Initialize the 'id' column with values
        cursor.execute("SELECT filename FROM research_papers")
        filenames = cursor.fetchall()
        for i, filename in enumerate(filenames):
            cursor.execute("UPDATE research_papers SET id = ? WHERE filename = ?", (i + 1, filename[0]))
        conn.commit()
        print("Added and initialized 'id' column to research_papers table.")


    # Fetch papers that don't have summaries
    cursor.execute("SELECT id, filename, cleaned_text FROM research_papers WHERE summary IS NULL OR summary = ''")
    papers = cursor.fetchall()

    total_papers = len(papers)
    print(f"Found {total_papers} papers to summarize.")

    for idx, (paper_id, filename, text) in enumerate(papers, 1):
        print(f"Processing {idx}/{total_papers}: {filename}")

        summary = generate_summary(text) if text else "Text too short or empty for summarization"

        # Update the database with the summary
        cursor.execute("UPDATE research_papers SET summary = ? WHERE id = ?", (summary, paper_id))
        conn.commit()
        print(f"Summary for {filename}: {summary[:100]}...")  # Print first 100 chars

    conn.close()
    print("Summarization complete. All summaries stored in the database.")

# Step 6: Verify the results
def verify_summaries():
    conn = connect_to_db()
    cursor = conn.cursor()

    cursor.execute("SELECT filename, title, summary FROM research_papers LIMIT 5")
    rows = cursor.fetchall()

    print("\nSample summaries from database:")
    for row in rows:
        print(f"Filename: {row[0]}")
        print(f"Title: {row[1]}")
        print(f"Summary: {row[2]}")
        print("-" * 50)

    conn.close()

# Run the script
if __name__ == "__main__":
    summarize_papers()
    verify_summaries()

Loading BERT summarization model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Added 'summary' column to research_papers table.
Added and initialized 'id' column to research_papers table.
Found 1074 papers to summarize.
Processing 1/1074: pubmed_40327459
Summary for pubmed_40327459: ...
Processing 2/1074: pubmed_40327390
Summary for pubmed_40327390: ...
Processing 3/1074: pubmed_40327384
Summary for pubmed_40327384: ...
Processing 4/1074: pubmed_40327329
Summary for pubmed_40327329: ...
Processing 5/1074: pubmed_40327286
27918725
33766750
17537614
20668052
33754023
19641587
35210368
17384583
9446642
32392088
34753510
35134463
33045622
24382354
21440622
23474486
35273493
30053525
38574433
37154056
26232232
33314649
32468629
11709088
37425037
23325925
10628339
36435202
32112883
24058702
31883358
38323433
26112889
17250813
37871668
33437153
38150302
38316068
29061338
27567406
35914558
37714227
36510036
32958254
29304479
Summary for pubmed_40327286
27918725
33766750
17537614
20668052
33754023
19641587
35210368
17384583
9446642
32392088
34753510
35134463
33045622
2438

#############################

week 2 old

##################################

In [ ]:
# pip install transformers torch  pandas

In [ ]:
"""
# week 2

import sqlite3
from transformers import pipeline
import pandas as pd

# Step 1: Set up the Hugging Face summarization model
print("Loading summarization model...")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Step 2: Connect to the SQLite database
def connect_to_db():
    conn = sqlite3.connect("healthcare_research.db")
    return conn

# Step 3: Add a summary column to the database (if not already present)
def add_summary_column(conn):
    cursor = conn.cursor()
    # Check if 'summary' column exists, if not, add it
    cursor.execute("PRAGMA table_info(research_papers)")
    columns = [col[1] for col in cursor.fetchall()]
    if "summary" not in columns:
        cursor.execute("ALTER TABLE research_papers ADD COLUMN summary TEXT")
        print("Added 'summary' column to research_papers table.")
    conn.commit()

# Step 4: Summarize text
def generate_summary(text, max_length=150, min_length=30):
    try:
        # Truncate text if too long (BART has a max input length of 1024 tokens)
        if len(text) > 1000:
            text = text[:1000]
        summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
        return summary
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return "Summary generation failed"

# Step 5: Process papers and update database
def summarize_papers():
    conn = connect_to_db()
    add_summary_column(conn)
    cursor = conn.cursor()

    # Fetch all papers (you can use 'abstract' or 'cleaned_text' for summarization)
    cursor.execute("SELECT id, filename, cleaned_text FROM research_papers WHERE summary IS NULL OR summary = ''")
    papers = cursor.fetchall()

    total_papers = len(papers)
    print(f"Found {total_papers} papers to summarize.")

    for idx, (paper_id, filename, text) in enumerate(papers, 1):
        print(f"Processing {idx}/{total_papers}: {filename}")

        if not text or len(text.strip()) < 50:  # Skip if text is too short or empty
            summary = "Text too short or empty for summarization"
        else:
            summary = generate_summary(text)

        # Update the database with the summary
        cursor.execute("UPDATE research_papers SET summary = ? WHERE id = ?", (summary, paper_id))
        conn.commit()
        print(f"Summary for {filename}: {summary[:100]}...")  # Print first 100 chars

    conn.close()
    print("Summarization complete. All summaries stored in the database.")

# Step 6: Verify the results
def verify_summaries():
    conn = connect_to_db()
    cursor = conn.cursor()

    cursor.execute("SELECT filename, title, summary FROM research_papers LIMIT 5")
    rows = cursor.fetchall()

    print("\nSample summaries from database:")
    for row in rows:
        print(f"Filename: {row[0]}")
        print(f"Title: {row[1]}")
        print(f"Summary: {row[2]}")
        print("-" * 50)

    conn.close()

# Run the script
if __name__ == "__main__":
    summarize_papers()
    verify_summaries()
    """

'\n# week 2\n\nimport sqlite3\nfrom transformers import pipeline\nimport pandas as pd\n\n# Step 1: Set up the Hugging Face summarization model\nprint("Loading summarization model...")\nsummarizer = pipeline("summarization", model="facebook/bart-large-cnn")\n\n# Step 2: Connect to the SQLite database\ndef connect_to_db():\n    conn = sqlite3.connect("healthcare_research.db")\n    return conn\n\n# Step 3: Add a summary column to the database (if not already present)\ndef add_summary_column(conn):\n    cursor = conn.cursor()\n    # Check if \'summary\' column exists, if not, add it\n    cursor.execute("PRAGMA table_info(research_papers)")\n    columns = [col[1] for col in cursor.fetchall()]\n    if "summary" not in columns:\n        cursor.execute("ALTER TABLE research_papers ADD COLUMN summary TEXT")\n        print("Added \'summary\' column to research_papers table.")\n    conn.commit()\n\n# Step 4: Summarize text\ndef generate_summary(text, max_length=150, min_length=30):\n    try:\n   

##################################################

week 3

################################################

BERTopic and visualization librarie

In [ ]:
!pip install bertopic pandas matplotlib seaborn wordcloud sqlite3
!pip install bertopic
!pip install bertopic matplotlib seaborn wordcloud pandas sqlite3
!pip install bertopic matplotlib seaborn wordcloud pandas

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
  Using cached bertopic-0.17.0-py3-none-any.whl.metadata (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:

^C
^C


In [ ]:
"""
import sqlite3
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Ensure Matplotlib uses a non-interactive backend
plt.switch_backend('agg')

# Step 1: Load Data from SQLite
def load_data_from_db():
    conn = sqlite3.connect("healthcare_research.db")
    query = "SELECT filename, cleaned_text FROM research_papers"
    df = pd.read_sql_query(query, conn)
    conn.close()
    print(f"Loaded {len(df)} records from the database.")
    print(f"Sample cleaned_text lengths: {df['cleaned_text'].str.len().head().tolist()}")
    return df

# Step 2: Topic Modeling with LDA
def perform_topic_modeling(df):
    docs = df["cleaned_text"].tolist()
    num_docs = len(docs)
    print(f"Number of documents: {num_docs}")

    if num_docs < 2:
        print("Too few documents for topic modeling (minimum 2 required).")
        return None, None, None

    # Vectorize the text
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
    doc_term_matrix = vectorizer.fit_transform(docs)
    feature_names = vectorizer.get_feature_names_out()

    # Fit LDA model
    n_topics = min(5, num_docs)  # Limit topics to number of documents
    print(f"Using {n_topics} topics for LDA.")
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(doc_term_matrix)

    # Get topic assignments for each document
    topic_assignments = lda.transform(doc_term_matrix).argmax(axis=1)
    df["topic"] = topic_assignments

    # Generate topic info (topic ID, keywords, count)
    topic_info = []
    for topic_idx in range(n_topics):
        top_words = [feature_names[i] for i in lda.components_[topic_idx].argsort()[:-10 - 1:-1]]
        num_docs_in_topic = (topic_assignments == topic_idx).sum()
        topic_info.append({
            "Topic": topic_idx,
            "Name": f"{topic_idx}_" + "_".join(top_words[:3]),
            "Keywords": ", ".join(top_words),
            "Count": num_docs_in_topic
        })
    topic_info_df = pd.DataFrame(topic_info)

    print("Topic modeling completed. Topic info:")
    print(topic_info_df.head(10))

    return lda, topic_info_df, df

# Step 3: Store Topic Results in SQLite
def store_topics_in_db(df, topic_info):
    conn = sqlite3.connect("healthcare_research.db")
    cursor = conn.cursor()

    # Drop the existing topics table to recreate with correct schema (or alter it)
    cursor.execute("DROP TABLE IF EXISTS topics")
    cursor.execute('''
        CREATE TABLE topics (
            topic_id INTEGER PRIMARY KEY,
            topic_name TEXT,
            keywords TEXT,
            num_documents INTEGER
        )
    ''')

    # Add topic_id column to research_papers if not exists
    try:
        cursor.execute('ALTER TABLE research_papers ADD COLUMN topic_id INTEGER')
    except sqlite3.OperationalError:
        print("Column topic_id already exists in research_papers, skipping alteration.")

    # Insert topic info
    for index, row in topic_info.iterrows():
        cursor.execute('''
            INSERT OR REPLACE INTO topics (topic_id, topic_name, keywords, num_documents)
            VALUES (?, ?, ?, ?)
        ''', (row["Topic"], row["Name"], row["Keywords"], row["Count"]))

    # Update research_papers with topic IDs
    for index, row in df.iterrows():
        if pd.notna(row["topic"]):
            cursor.execute('''
                UPDATE research_papers SET topic_id = ? WHERE filename = ?
            ''', (int(row["topic"]), row["filename"]))

    conn.commit()
    conn.close()
    print("Topic modeling results stored in database.")

# Step 4: Visualizations
def create_visualizations(topic_info):
    valid_topics = topic_info[topic_info["Count"] > 0]  # Exclude empty topics
    if valid_topics.empty:
        print("No valid topics found. No visualizations created.")
        return

    # Bar chart
    plt.figure(figsize=(10, 6))
    sns.barplot(x="Count", y="Name", data=valid_topics.head(10))
    plt.title("Top 10 Most Common Topics")
    plt.xlabel("Number of Documents")
    plt.ylabel("Topic Name")
    plt.tight_layout()
    plt.savefig("top_topics_bar_chart.png")
    print("Saved bar chart to 'top_topics_bar_chart.png'")
    plt.close()

    # Word clouds
    top_topics = valid_topics.head(5)["Topic"].tolist()
    for topic_id in top_topics:
        keywords = valid_topics[valid_topics["Topic"] == topic_id]["Keywords"].iloc[0].split(", ")
        word_dict = {word: 1.0 / (i + 1) for i, word in enumerate(keywords)}  # Simple weighting
        wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(word_dict)
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
        plt.title(f"Word Cloud for Topic {topic_id}: {valid_topics[valid_topics['Topic'] == topic_id]['Name'].iloc[0]}")
        plt.savefig(f"wordcloud_topic_{topic_id}.png")
        print(f"Saved word cloud for Topic {topic_id} to 'wordcloud_topic_{topic_id}.png'")
        plt.close()

# Main execution
if __name__ == "__main__":
    df = load_data_from_db()
    if df.empty:
        print("No data found in the database. Please ensure the research_papers table is populated.")
    else:
        lda_model, topic_info, df_with_topics = perform_topic_modeling(df)
        if lda_model is None:
            print("Topic modeling failed. Check error messages above.")
        else:
            store_topics_in_db(df_with_topics, topic_info)
            create_visualizations(topic_info)
  """

Loaded 106 records from the database.
Sample cleaned_text lengths: [74782, 26401, 44194, 14468, 38267]
Number of documents: 106
Using 5 topics for LDA.
Topic modeling completed. Topic info:
   Topic                           Name  \
0      0         0_public_care_services   
1      1        1_waste_medical_patient   
2      2            2_pain_mg_treatment   
3      3     3_countries_mortality_data   
4      4  4_information_hispanic_states   

                                            Keywords  Count  
0  public, care, services, al, et, policy, global...     32  
1  waste, medical, patient, hospitals, operating,...     10  
2  pain, mg, treatment, patients, patient, manage...     30  
3  countries, mortality, data, years, age, deaths...     24  
4  information, hispanic, states, united, table, ...     10  
Column topic_id already exists in research_papers, skipping alteration.
Topic modeling results stored in database.
Saved bar chart to 'top_topics_bar_chart.png'
Saved word cloud f

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Dummy data
data = np.random.rand(10, 12)

plt.figure(figsize=(8, 6))
sns.heatmap(data, annot=True, fmt=".2f", cmap='YlGnBu')
plt.title("Sample Heatmap")
plt.tight_layout()
plt.show()
sns.heatmap(data, annot=True, fmt=".2f", cmap='YlGnBu')
plt.title("Sample Heatmap")
plt.tight_layout()
plt.show()


week 3 new with network/heat

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
from scipy.sparse import csr_matrix
from datetime import datetime

def create_advanced_visualizations(df_with_topics, topic_info):
    # Assign random dates if not already present
    if 'date' not in df_with_topics.columns:
        df_with_topics['date'] = pd.to_datetime(np.random.choice(
            pd.date_range('2020-01-01', '2025-03-30', freq='M'),
            size=len(df_with_topics)
        ))

    # Ensure consistent topic indexing
    topics = sorted(df_with_topics['topic'].unique())
    topic_to_idx = {topic: idx for idx, topic in enumerate(topics)}
    co_occurrence = np.zeros((len(topics), len(topics)))

    # Compute co-occurrence matrix
    for i, topic_i in enumerate(topics):
        for j, topic_j in enumerate(topics):
            if i <= j:
                keywords_list = topic_info[topic_info['Topic'] == topic_j]['Keywords']
                if not keywords_list.empty:
                    keywords = keywords_list.iloc[0].split(', ')
                    co_occurrence[i, j] = len(df_with_topics[
                        (df_with_topics['topic'] == topic_i) &
                        (df_with_topics['cleaned_text'].str.contains('|'.join(keywords)))
                    ])
                co_occurrence[j, i] = co_occurrence[i, j]

    # === 1. Heatmap ===
    plt.figure(figsize=(10, 8))
    sns.heatmap(co_occurrence, annot=True, fmt='.0f', cmap='YlGnBu',
                xticklabels=[f"Topic {t}" for t in topics],
                yticklabels=[f"Topic {t}" for t in topics])
    plt.title("Topic Co-occurrence Heatmap")
    plt.tight_layout()
    plt.savefig("topic_cooccurrence_heatmap.png")
    plt.show()

    # === 2. Bubble Plot Over Time ===
    df_with_topics['year'] = df_with_topics['date'].dt.year
    topic_time = df_with_topics.groupby(['year', 'topic']).size().reset_index(name='count')
    topic_time = topic_time.merge(topic_info[['Topic', 'Name']], left_on='topic', right_on='Topic')

    plt.figure(figsize=(12, 8))
    bubble_size = 100 * (topic_time['count'] / topic_time['count'].max())
    scatter = plt.scatter(topic_time['year'], topic_time['Name'],
                          s=bubble_size, alpha=0.5, c=topic_time['topic'], cmap='viridis')
    plt.colorbar(scatter, label='Topic ID')
    plt.xlabel("Year")
    plt.ylabel("Topic Name")
    plt.title("Topic Importance Over Time (Bubble Size = Document Count)")
    plt.tight_layout()
    plt.savefig("topic_importance_bubble.png")
    plt.show()

    # === 3. Network Graph ===
    G = nx.Graph()
    for _, row in topic_info.iterrows():
        G.add_node(row['Topic'], size=row['Count'], label=row['Name'])

    threshold = np.percentile(co_occurrence.flatten(), 75)
    for i, topic_i in enumerate(topics):
        for j, topic_j in enumerate(topics):
            if i < j and co_occurrence[i, j] > threshold:
                G.add_edge(topic_i, topic_j, weight=co_occurrence[i, j])

    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G, k=0.5)
    sizes = [G.nodes[n]['size'] * 20 for n in G.nodes]
    nx.draw_networkx_nodes(G, pos, node_size=sizes, node_color='skyblue', alpha=0.7)
    nx.draw_networkx_edges(G, pos, width=[d['weight']/10 for (u, v, d) in G.edges(data=True)], alpha=0.5)
    nx.draw_networkx_labels(G, pos, labels={n: G.nodes[n]['label'] for n in G.nodes}, font_size=8)

    # Optional edge labels (uncomment if desired)
    # edge_labels = {(u, v): f"{d['weight']:.0f}" for u, v, d in G.edges(data=True)}
    # nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)

    plt.title("Network Graph of Topic Relationships")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("topic_relationships_network.png")
    plt.show()


# # Ensure Matplotlib uses a non-interactive backend
# plt.switch_backend('agg')

# # Step 5: Additional Visualizations
# def create_advanced_visualizations(df_with_topics, topic_info):
#     # Assuming filename might contain a date or we need to simulate dates
#     # For this example, we'll simulate dates if not available
#     if 'date' not in df_with_topics.columns:
#         df_with_topics['date'] = pd.to_datetime(np.random.choice(
#             pd.date_range('2020-01-01', '2025-03-30', freq='M'),
#             size=len(df_with_topics)
#         ))

#     # --- Heatmap: Topic Co-occurrence ---
#     # Create a co-occurrence matrix
#     topics = df_with_topics['topic'].unique()
#     co_occurrence = np.zeros((len(topics), len(topics)))
#     for i, topic_i in enumerate(topics):
#         for j, topic_j in enumerate(topics):
#             if i <= j:  # Only calculate upper triangle to avoid duplication
#                 co_occurrence[i, j] = len(df_with_topics[
#                     (df_with_topics['topic'] == topic_i) &
#                     (df_with_topics['cleaned_text'].str.contains('|'.join(
#                         topic_info[topic_info['Topic'] == topic_j]['Keywords'].iloc[0].split(', ')
#                     )))
#                 ])
#                 co_occurrence[j, i] = co_occurrence[i, j]  # Symmetric matrix

#     plt.figure(figsize=(10, 8))
#     sns.heatmap(co_occurrence, annot=True, fmt='.0f', cmap='YlGnBu',
#                 xticklabels=[f"Topic {t}" for t in topics],
#                 yticklabels=[f"Topic {t}" for t in topics])
#     plt.title("Topic Co-occurrence Heatmap")
#     plt.tight_layout()
#     plt.savefig("topic_cooccurrence_heatmap.png")
#     print("Saved heatmap to 'topic_cooccurrence_heatmap.png'")
#     plt.close()

#     # --- Bubble Chart: Topic Importance Over Time ---
#     # Aggregate topic counts by year
#     df_with_topics['year'] = df_with_topics['date'].dt.year
#     topic_time = df_with_topics.groupby(['year', 'topic']).size().reset_index(name='count')
#     topic_time = topic_time.merge(topic_info[['Topic', 'Name']], left_on='topic', right_on='Topic')

#     plt.figure(figsize=(12, 8))
#     plt.scatter(topic_time['year'], topic_time['Name'], s=topic_time['count']*100,
#                 alpha=0.5, c=topic_time['topic'], cmap='viridis')
#     plt.colorbar(label='Topic ID')
#     plt.xlabel("Year")
#     plt.ylabel("Topic Name")
#     plt.title("Topic Importance Over Time (Bubble Size = Document Count)")
#     plt.tight_layout()
#     plt.savefig("topic_importance_bubble.png")
#     print("Saved bubble chart to 'topic_importance_bubble.png'")
#     plt.close()

#     # --- Network Graph: Topic Relationships ---
#     G = nx.Graph()
#     # Add nodes (topics)
#     for _, row in topic_info.iterrows():
#         G.add_node(row['Topic'], size=row['Count'], label=row['Name'])

#     # Add edges based on co-occurrence (threshold to reduce clutter)
#     threshold = np.percentile(co_occurrence.flatten(), 75)  # Top 25% of co-occurrences
#     for i, topic_i in enumerate(topics):
#         for j, topic_j in enumerate(topics):
#             if i < j and co_occurrence[i, j] > threshold:
#                 G.add_edge(topic_i, topic_j, weight=co_occurrence[i, j])

#     plt.figure(figsize=(12, 10))
#     pos = nx.spring_layout(G, k=0.5)
#     sizes = [G.nodes[n]['size'] * 20 for n in G.nodes]
#     nx.draw_networkx_nodes(G, pos, node_size=sizes, node_color='skyblue', alpha=0.7)
#     nx.draw_networkx_edges(G, pos, width=[d['weight']/10 for (u, v, d) in G.edges(data=True)],
#                            alpha=0.5)
#     nx.draw_networkx_labels(G, pos, labels={n: G.nodes[n]['label'] for n in G.nodes},
#                             font_size=8)
#     plt.title("Network Graph of Topic Relationships")
#     plt.axis('off')
#     plt.tight_layout()
#     plt.savefig("topic_relationships_network.png")
#     print("Saved network graph to 'topic_relationships_network.png'")
#     plt.close()

# # Update main execution to include new visualizations
# if __name__ == "__main__":
#     df = load_data_from_db()
#     if df.empty:
#         print("No data found in the database. Please ensure the research_papers table is populated.")
#     else:
#         lda_model, topic_info, df_with_topics = perform_topic_modeling(df)
#         if lda_model is None:
#             print("Topic modeling failed. Check error messages above.")
#         else:
#             store_topics_in_db(df_with_topics, topic_info)
#             create_visualizations(topic_info)
#             create_advanced_visualizations(df_with_topics, topic_info)

week 3 new 3

In [ ]:
# Install required packages
!pip install bertopic pandas matplotlib seaborn wordcloud sqlite3
!pip install scikit-learn  # For TF-IDF and other utilities

import sqlite3
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from bertopic import BERTopic
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import networkx as nx
from datetime import datetime

# Ensure Matplotlib uses a non-interactive backend
plt.switch_backend('agg')

# Step 1: Load Data from SQLite
def load_data_from_db():
    conn = sqlite3.connect("healthcare_research.db")
    query = "SELECT filename, cleaned_text FROM research_papers"
    df = pd.read_sql_query(query, conn)
    conn.close()
    print(f"Loaded {len(df)} records from the database.")
    print(f"Sample cleaned_text lengths: {df['cleaned_text'].str.len().head().tolist()}")
    return df

# Step 2a: Topic Modeling with Improved LDA (using TF-IDF)
def perform_lda_topic_modeling(df):
    docs = df["cleaned_text"].dropna().tolist()  # Drop NaN values
    num_docs = len(docs)
    print(f"Number of documents for LDA: {num_docs}")

    if num_docs < 2:
        print("Too few documents for LDA (minimum 2 required).")
        return None, None, None

    # Use TF-IDF instead of CountVectorizer
    vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
    doc_term_matrix = vectorizer.fit_transform(docs)
    feature_names = vectorizer.get_feature_names_out()

    # Fit LDA model
    n_topics = min(5, num_docs)
    print(f"Using {n_topics} topics for LDA.")
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(doc_term_matrix)

    # Get topic assignments
    topic_assignments = lda.transform(doc_term_matrix).argmax(axis=1)
    df.loc[df["cleaned_text"].notna(), "lda_topic"] = topic_assignments

    # Generate topic info
    topic_info = []
    for topic_idx in range(n_topics):
        top_words = [feature_names[i] for i in lda.components_[topic_idx].argsort()[:-10 - 1:-1]]
        num_docs_in_topic = (topic_assignments == topic_idx).sum()
        topic_info.append({
            "Topic": topic_idx,
            "Name": f"{topic_idx}_" + "_".join(top_words[:3]),
            "Keywords": ", ".join(top_words),
            "Count": num_docs_in_topic
        })
    topic_info_df = pd.DataFrame(topic_info)
    print("LDA Topic modeling completed. Topic info:")
    print(topic_info_df.head(10))
    return lda, topic_info_df, df

# Step 2b: Topic Modeling with BERTopic
def perform_bertopic_modeling(df):
    docs = df["cleaned_text"].dropna().tolist()
    num_docs = len(docs)
    print(f"Number of documents for BERTopic: {num_docs}")

    if num_docs < 2:
        print("Too few documents for BERTopic (minimum 2 required).")
        return None, None

    # Initialize and fit BERTopic
    topic_model = BERTopic(nr_topics="auto", min_topic_size=2, verbose=True)
    topics, _ = topic_model.fit_transform(docs)
    df.loc[df["cleaned_text"].notna(), "bertopic"] = topics

    # Get topic info
    topic_info = topic_model.get_topic_info()
    topic_info = topic_info.rename(columns={"Topic": "Topic", "Count": "Count", "Name": "Name"})
    topic_info["Keywords"] = topic_info["Topic"].apply(
        lambda x: ", ".join([word for word, _ in topic_model.get_topic(x)[:10]]) if x >= 0 else "N/A"
    )
    print("BERTopic modeling completed. Topic info:")
    print(topic_info.head(10))
    return topic_model, topic_info, df

# Step 2c: Dynamic Topic Modeling (LDA over time)
def perform_dynamic_topic_modeling(df):
    docs = df["cleaned_text"].dropna().tolist()
    if 'date' not in df.columns:
        df['date'] = pd.to_datetime(np.random.choice(
            pd.date_range('2020-01-01', '2025-03-30', freq='M'),
            size=len(df)
        ))
    df['year'] = df['date'].dt.year

    dynamic_results = []
    for year in sorted(df['year'].unique()):
        year_docs = df[df['year'] == year]["cleaned_text"].dropna().tolist()
        if len(year_docs) < 2:
            continue
        vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
        doc_term_matrix = vectorizer.fit_transform(year_docs)
        n_topics = min(5, len(year_docs))
        lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
        lda.fit(doc_term_matrix)
        topic_dist = lda.transform(doc_term_matrix).sum(axis=0)
        dynamic_results.append({"Year": year, "Topic_Distribution": topic_dist / topic_dist.sum()})

    dynamic_df = pd.DataFrame(dynamic_results)
    print("Dynamic topic modeling completed.")
    return dynamic_df, df

# Step 3: Store Results in SQLite
def store_results_in_db(df, lda_info, bertopic_info):
    conn = sqlite3.connect("healthcare_research.db")
    cursor = conn.cursor()

    # Update topics table for LDA
    cursor.execute("DROP TABLE IF EXISTS lda_topics")
    cursor.execute('''
        CREATE TABLE lda_topics (
            topic_id INTEGER PRIMARY KEY,
            topic_name TEXT,
            keywords TEXT,
            num_documents INTEGER
        )
    ''')
    for _, row in lda_info.iterrows():
        cursor.execute('''
            INSERT INTO lda_topics (topic_id, topic_name, keywords, num_documents)
            VALUES (?, ?, ?, ?)
        ''', (row["Topic"], row["Name"], row["Keywords"], row["Count"]))

    # Create table for BERTopic
    cursor.execute("DROP TABLE IF EXISTS bertopic_topics")
    cursor.execute('''
        CREATE TABLE bertopic_topics (
            topic_id INTEGER PRIMARY KEY,
            topic_name TEXT,
            keywords TEXT,
            num_documents INTEGER
        )
    ''')
    for _, row in bertopic_info.iterrows():
        if row["Topic"] >= 0:  # Exclude -1 (outliers in BERTopic)
            cursor.execute('''
                INSERT INTO bertopic_topics (topic_id, topic_name, keywords, num_documents)
                VALUES (?, ?, ?, ?)
            ''', (row["Topic"], row["Name"], row["Keywords"], row["Count"]))

    # Update research_papers with topic IDs
    try:
        cursor.execute('ALTER TABLE research_papers ADD COLUMN lda_topic_id INTEGER')
        cursor.execute('ALTER TABLE research_papers ADD COLUMN bertopic_id INTEGER')
    except sqlite3.OperationalError:
        pass  # Columns might already exist

    for _, row in df.iterrows():
        if pd.notna(row.get("lda_topic")):
            cursor.execute('UPDATE research_papers SET lda_topic_id = ? WHERE filename = ?',
                           (int(row["lda_topic"]), row["filename"]))
        if pd.notna(row.get("bertopic")) and row["bertopic"] >= 0:
            cursor.execute('UPDATE research_papers SET bertopic_id = ? WHERE filename = ?',
                           (int(row["bertopic"]), row["filename"]))

    conn.commit()
    conn.close()
    print("Results stored in database.")

# Step 4: Visualizations (Updated for all models)
def create_visualizations(lda_info, bertopic_info, dynamic_df):
    # LDA Bar Chart
    plt.figure(figsize=(10, 6))
    sns.barplot(x="Count", y="Name", data=lda_info.head(10))
    plt.title("Top 10 LDA Topics")
    plt.savefig("lda_top_topics.png")
    plt.close()

    # BERTopic Bar Chart
    valid_topics = bertopic_info[bertopic_info["Topic"] >= 0]
    plt.figure(figsize=(10, 6))
    sns.barplot(x="Count", y="Name", data=valid_topics.head(10))
    plt.title("Top 10 BERTopic Topics")
    plt.savefig("bertopic_top_topics.png")
    plt.close()

    # Dynamic Topic Evolution
    if not dynamic_df.empty:
        plt.figure(figsize=(12, 6))
        for i in range(dynamic_df["Topic_Distribution"].iloc[0].shape[0]):
            plt.plot(dynamic_df["Year"], dynamic_df["Topic_Distribution"].apply(lambda x: x[i]),
                     label=f"Topic {i}")
        plt.title("Dynamic Topic Evolution Over Time")
        plt.xlabel("Year")
        plt.ylabel("Topic Prevalence")
        plt.legend()
        plt.savefig("dynamic_topic_evolution.png")
        plt.close()
        print("Saved dynamic topic evolution plot.")

# Main execution
if __name__ == "__main__":
    df = load_data_from_db()
    if df.empty:
        print("No data found in the database.")
    else:
        # LDA with TF-IDF
        lda_model, lda_info, df = perform_lda_topic_modeling(df)

        # BERTopic
        bertopic_model, bertopic_info, df = perform_bertopic_modeling(df)

        # Dynamic Topic Modeling
        dynamic_df, df = perform_dynamic_topic_modeling(df)

        if lda_model and bertopic_model:
            store_results_in_db(df, lda_info, bertopic_info)
            create_visualizations(lda_info, bertopic_info, dynamic_df)
        else:
            print("One or more topic modeling steps failed.")

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
Loaded 1074 records from the database.
Sample cleaned_text lengths: [1727, 1937, 2275, 1640, 1636]
Number of documents for LDA: 1074
Using 5 topics for LDA.


2025-05-07 00:56:26,953 - BERTopic - Embedding - Transforming documents to embeddings.


LDA Topic modeling completed. Topic info:
   Topic                            Name  \
0      0  0_healthcare_innovation_health   
1      1                1_sha_laura_miao   
2      2     2_forces_sections_following   
3      3    3_celebration_yunan_michigan   
4      4      4_lawton_greenhalgh_trisha   

                                            Keywords  Count  
0  healthcare, innovation, health, care, data, in...   1072  
1  sha, laura, miao, qu, shu, xing, films, datta,...      0  
2  forces, sections, following, col, redefining, ...      1  
3  celebration, yunan, michigan, arbor, ann, beth...      0  
4  lawton, greenhalgh, trisha, burns, testosteron...      1  
Number of documents for BERTopic: 1074


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

2025-05-07 00:58:02,297 - BERTopic - Embedding - Completed ✓
2025-05-07 00:58:02,299 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-07 00:58:16,722 - BERTopic - Dimensionality - Completed ✓
2025-05-07 00:58:16,724 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-07 00:58:16,797 - BERTopic - Cluster - Completed ✓
2025-05-07 00:58:16,798 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-05-07 00:58:17,404 - BERTopic - Representation - Completed ✓
2025-05-07 00:58:17,405 - BERTopic - Topic reduction - Reducing number of topics
2025-05-07 00:58:17,435 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-07 00:58:17,950 - BERTopic - Representation - Completed ✓
2025-05-07 00:58:17,953 - BERTopic - Topic reduction - Reduced number of topics from 99 to 34


BERTopic modeling completed. Topic info:
   Topic  Count                                     Name  \
0     -1    170                         -1_and_the_of_to   
1      0    366                          0_the_and_of_to   
2      1    320                                    1____   
3      2     47            2_health_disease_public_years   
4      3     30                                    3____   
5      4     21          4_blockchain_the_and_technology   
6      5     12  5_flexible_wearable_electronics_sensors   
7      6     10                    6_iot_the_and_devices   
8      7      9                    7_of_the_education_in   
9      8      8               8_hydronephrosis_ai_and_of   

                                      Representation  \
0  [and, the, of, to, in, waste, healthcare, for,...   
1  [the, and, of, to, in, innovation, healthcare,...   
2                               [, , , , , , , , , ]   
3  [health, disease, public, years, care, global,...   
4                 

tranfering image to database

# New Section

In [ ]:
import sqlite3
import os

# List of PNG files to store in the database
png_files = [
    "bertopic_top_topics.png",  # N
    "dynamic_topic_evolution.png",
    "lda_top_topics.png",  # Corrected from "Ida_top_topics.png" assuming it's a typo
    "top_topics_bar_chart.png",
    "topic_cooccurrence_heatmap.png",
    "topic_importance_bubble.png",
    "topic_relationships_network.png",
    "wordcloud_topic_0.png",  # Corrected "wordcloud_topic_o.png" to "wordcloud_topic_0.png" assuming typo
    "wordcloud_topic_1.png",
    "wordcloud_topic_2.png",  # Corrected "wordcloud_topic_2png" to "wordcloud_topic_2.png"
    "wordcloud_topic_3.png",
    "wordcloud_topic_4.png",
    "research_papers_table.png"
]

# Function to store PNG files in the database
def store_pngs_in_db(png_files, db_name="healthcare_research.db"):
    # Connect to the database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create a table for storing images if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS visualizations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT UNIQUE,
            image_data BLOB
        )
    ''')

    # Store each PNG file
    for png_file in png_files:
        if os.path.exists(png_file):
            try:
                with open(png_file, 'rb') as file:
                    image_data = file.read()
                cursor.execute('''
                    INSERT OR REPLACE INTO visualizations (filename, image_data)
                    VALUES (?, ?)
                ''', (png_file, sqlite3.Binary(image_data)))
                print(f"Stored {png_file} in the database.")
            except Exception as e:
                print(f"Error storing {png_file}: {e}")
        else:
            print(f"File {png_file} not found. Skipping.")

    # Commit changes and close connection
    conn.commit()
    conn.close()
    print("All available PNG files have been stored in the database.")

# Execute the function
if __name__ == "__main__":
    store_pngs_in_db(png_files)

Stored bertopic_top_topics.png in the database.
Stored dynamic_topic_evolution.png in the database.
Stored lda_top_topics.png in the database.
File top_topics_bar_chart.png not found. Skipping.
File topic_cooccurrence_heatmap.png not found. Skipping.
File topic_importance_bubble.png not found. Skipping.
File topic_relationships_network.png not found. Skipping.
File wordcloud_topic_0.png not found. Skipping.
File wordcloud_topic_1.png not found. Skipping.
File wordcloud_topic_2.png not found. Skipping.
File wordcloud_topic_3.png not found. Skipping.
File wordcloud_topic_4.png not found. Skipping.
Stored research_papers_table.png in the database.
All available PNG files have been stored in the database.
